<a href="https://colab.research.google.com/github/AdityaSadana/Text-Generator/blob/main/Text_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
response=requests.get("http://www.shakespeares-sonnets.com/sonnet/all.php")

In [2]:
from bs4 import BeautifulSoup
soup=BeautifulSoup(response.content)

In [3]:
text=""
for row in soup.findAll('p'):
  text+=row.text

In [4]:
import string

def remove_punc(text):
  for j in string.punctuation:
    text=text.replace(j," ")

  return text.replace("\n"," ")

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stopwords=stopwords.words("english")
lemmatizer=WordNetLemmatizer()
words=[lemmatizer.lemmatize(word.lower()) for word in word_tokenize(remove_punc(text)) if word not in stopwords]

len(words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


11711

In [6]:
word_mapper={}
index_mapper={}
i=0
for word in words:
  if word not in word_mapper:
    word_mapper[word]=i
    index_mapper[i]=word
    i+=1

len(word_mapper)

2963

In [7]:
encoded_text=[]
for word in words:
  encoded_text.append(word_mapper[word])

In [8]:
train_x=[]
train_y=[]

length=5
for i in range(length,len(encoded_text)):
  train_x.append(encoded_text[i-length:i])
  train_y.append(encoded_text[i])

In [9]:
import numpy as np
from tensorflow.keras.utils import to_categorical

vocab_size=len(word_mapper)
train_x=np.array(train_x)
train_y=np.array(train_y)

train_x.shape, train_y.shape, train_x[0], train_y[0]

((11706, 5), (11706,), array([0, 1, 2, 3, 4]), 5)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding

model=Sequential()
model.add(Embedding(vocab_size, 50, input_length=length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 50)             148150    
                                                                 
 lstm (LSTM)                 (None, 5, 100)            60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 2963)              299263    
                                                                 
Total params: 598,313
Trainable params: 598,313
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=['accuracy'])
model.fit(train_x, train_y,epochs=100)

Epoch 1/100
366/366 [==============================] - 9s 7ms/step - loss: 7.2730 - accuracy: 0.0278
Epoch 2/100
366/366 [==============================] - 2s 6ms/step - loss: 6.9005 - accuracy: 0.0294
Epoch 3/100
366/366 [==============================] - 2s 6ms/step - loss: 6.8476 - accuracy: 0.0299
Epoch 4/100
366/366 [==============================] - 2s 6ms/step - loss: 6.7900 - accuracy: 0.0286
Epoch 5/100
366/366 [==============================] - 2s 6ms/step - loss: 6.6868 - accuracy: 0.0301
Epoch 6/100
366/366 [==============================] - 2s 7ms/step - loss: 6.5860 - accuracy: 0.0287
Epoch 7/100
366/366 [==============================] - 2s 6ms/step - loss: 6.4822 - accuracy: 0.0302
Epoch 8/100
366/366 [==============================] - 2s 6ms/step - loss: 6.3530 - accuracy: 0.0327
Epoch 9/100
366/366 [==============================] - 2s 6ms/step - loss: 6.2118 - accuracy: 0.0332
Epoch 10/100
366/366 [==============================] - 2s 6ms/step - loss: 6.0523 - accura

In [20]:
input_text="And his love-kindling fire did quickly"
encoded_text=[]

clean_text=[lemmatizer.lemmatize(word.lower()) for word in word_tokenize(remove_punc(input_text)) if word not in stopwords]
print(clean_text)
for word in clean_text:
  if word in word_mapper:
    encoded_text.append(word_mapper[word])

if len(encoded_text)>=length: encoded_text=encoded_text[-length:]
else: encoded_text=[vocab_size]*(length-len(encoded_text))+encoded_text

['and', 'love', 'kindling', 'fire', 'quickly']


In [21]:
pred=model.predict(np.array([encoded_text]))
word_index=np.argmax(pred)

In [22]:
index_mapper[word_index]

'steep'

In [23]:
model.save("Text-generator.h5")

In [25]:
import pickle

file=open("word-mapper.pkl","wb")
pickle.dump(word_mapper,file)
file=open("index-mapper.pkl","wb")
pickle.dump(index_mapper,file)